### MATH50003 Week 3 Notes

In [5]:
using LinearAlgebra, Plots, BenchmarkTools, ColorBitstring, Test
import Base: getindex, setindex!, size, *, \

In [6]:
# Vector
A = [1 2; 3 4; 5 6]
vec(A)
A' # Transpose

2×3 adjoint(::Matrix{Int64}) with eltype Int64:
 1  3  5
 2  4  6

In [7]:
# Matrix multiplicationn in terms of algebraic operations
# A is a m by n matrix, and x is n vector (n by 1)

# go row-by-row
function mul_rows(A, x)
    m, n = size(A)
    c = zeros(eltype(x), m) #eltype: type of the elements of a vector/matrix; zeros() creates a vector contain 0 only
    for k = 1:m, j = 1:n
        c[k] += A[k, j] * x[j]
    end
    c
end

# go col-by-col, significantly faster than mul_rows due to accessing memory of A in order
function mul(A, x)
    m, n = size(A)
    c = zeros(eltype(x), m)
    for j = 1:n, k = 1:m
        c[k] += A[k, j] * x[j]
    end
    c
end

# Test
n = 1000
A = randn(n, n)
x = randn(n)

@btime mul_rows(A,x)
@btime mul(A,x)
@btime A*x # Built-in high performance function

  2.558 ms (1 allocation: 7.94 KiB)
  798.635 μs (1 allocation: 7.94 KiB)
  97.382 μs (1 allocation: 7.94 KiB)


1000-element Vector{Float64}:
  27.369507139809077
  36.35603806855054
   7.756548658282468
 -12.925541515616569
  13.480294065804506
 -35.43636406678427
  21.764616992867396
   7.859309200293712
 -34.52377233501758
  22.04509016660268
  -8.40052831570872
  49.35232373858988
  65.83654441600814
   ⋮
  59.88674972890512
  79.43032092909385
  65.23893569113032
 -40.21488501113795
   7.355771026314127
  10.877594009266256
   3.0548887487119476
  19.7909282421017
 -86.48245349420947
  63.05208534982382
 -23.695475090917398
  30.761209816890826

In [8]:
# The following code finds examples showing A*x is not implemented as mul(A,x), since floating bits don't match for the two methods.
function findmuldifference(n, l) # n: size, l: number of attempts
    for i = 1:l
        A = randn(n, n)
        x = randn(n)
        if A*x != mul(A,x)
            return (A,x)
        end
    end
end

n = 10
l = 100
A, x = findmuldifference(n, l)
println("Bits obtained by A*x")
printlnbits.(A*x)
println("Bits obtained by mul(A,x)")
printlnbits.(mul(A,x))
println("Difference in bits")
printlnbits.(A*x - mul(A,x))

Bits obtained by A*x
1100000000001100110000111000000100001010111011111100001000101110
0100000000100011010001101010111000000010010101100001010011110111
0100000000000100101000110011001100111101111011010100111100111000
1011111111110000011101111111101101111001010101110100011010000111
0100000000001000001111100011100101100000111110000010010010101100
1100000000001011111011110110011101010000001001110111100100000011
0011111111101010001110110101111010101101100010000001110001100100
0100000000000111111111001110101000101010001001010110110111101011
0100000000011111000001000100111010011011010110111011111011010001
0100000000010101011000111011000100010001101011111000011111010001
Bits obtained by mul(A,x)
1100000000001100110000111000000100001010111011111100001000101111
0100000000100011010001101010111000000010010101100001010011111000
0100000000000100101000110011001100111101111011010100111100111001
1011111111110000011101111111101101111001010101110100011010000111
0100000000001000001111100011100101100000111

10-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [9]:
# Solve linear system
# Ax = b is solve by A \ b

A = [1 2 3; 1 2 4; 3 7 8]
b = [10; 11; 12]
x = A \ b

A*x ≈ b

true

In [10]:
# Triangular matrices
A = [1 2 3; 4 5 6; 7 8 9]

U, L = UpperTriangular(A), LowerTriangular(A)
L # L stores all the data, but only show the lower triangular part, access all data by L.data.

3×3 LowerTriangular{Int64, Matrix{Int64}}:
 1  ⋅  ⋅
 4  5  ⋅
 7  8  9

In [11]:
# Implementation of forward / backward substitution with triangular system

function ldiv(U::UpperTriangular, b) # backward substitution for UpperTriangular multiplicationn, starting from last row.
    n = size(U, 1)

    if length(b) != n
        error("The system is not compatible.")
    end

    x = zeros(n)
    for k = n:-1:1
        r = b[k]
        for j = k+1:n
            r -= U[k, j] * x[j]
        end
        x[k] = r/U[k,k]
    end
    x
end

function ldiv(U::LowerTriangular, b) # forward substitution for LowerTriangular multiplicationn, starting from 1st row.
    n = size(U,1)

    if length(b) != n
        error("The system is not compatible.")
    end

    x = zeros(n)
    for k = 1:n
        r = b[k]
        for j = 1:k-1
            r -= U[k, j] * x[j]
        end
        x[k] = r /U[k,k]
    end
    x
end

ldiv (generic function with 2 methods)

### Banded matrices

*Definition:* Zero off a prescribed nunmber of diagonals.

*Bandwidths:* The number of non-zero diagonals.

More specifically, a matrix has strictly lower-bandwidth $l$ if it has lower-bandwidth $l$, and there exists a $j$ such that $A[j+l,j] \neq 0.$. A matrix has strictly upper-bandwidth $u$ if it has upper-bandwidth $u$ and there exists a $k$ such that $A[k, k+u] \neq 0.$

Intuitively, all the nonzero entries of the matrix are within K steps of the diagonal.



In [12]:
# Diagonal matrices are square matrices with bandwidths l=u=0
x = [1,2,3]
D = Diagonal(x) # multiplications involving diagonal matrices are in O(n) operations.


3×3 Diagonal{Int64, Vector{Int64}}:
 1  ⋅  ⋅
 ⋅  2  ⋅
 ⋅  ⋅  3

In [13]:
# Lower-bidiagonal matrix: square matrix has bandwidth (l,u) = (1,0)
# Upper-bidiagonal matrix: square matrix has bandwidth (l,u) = (0,1)

Bidiagonal([1,2,3], [4,5], :L)
Bidiagonal([1,2,3], [4,5], :U)

3×3 Bidiagonal{Int64, Vector{Int64}}:
 1  4  ⋅
 ⋅  2  5
 ⋅  ⋅  3

In [14]:
# Tridiagonal matrix: suqare matrix with bandwidths l=u=1
Tridiagonal([1,2], [3,4,5], [6,7])

3×3 Tridiagonal{Int64, Vector{Int64}}:
 3  6  ⋅
 1  4  7
 ⋅  2  5

### Question

Impelment an upper triangular matrix with bandwidth (l,u) = (0,2), and its * and \ operations which only take O(n) operations.

In [15]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]
    if j == k+2
        return U.du2[k]
    elseif j == k+1
        return U.du[k]
    elseif j == k
        return U.d[k]
    else
        return zero(eltype(U))
    end
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end

    # TODO: modify d,du,du2 so that U[k,j] == v
    if j == k+2
        U.du2[k] = v
    elseif j == k+1
        U.du[k] = v
    elseif j == k
        U.d[k] = v
    end
    U # by convention we return the matrix
end

A = UpperTridiagonal([1,2,3,4],[1,2,3],[1,2])

4×4 UpperTridiagonal{Int64}:
 1  1  1  0
 0  2  2  2
 0  0  3  3
 0  0  0  4

In [16]:
A[2,2]

2

In [17]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    # TODO: populate b so that U*x == b (up to rounding)
    n = size(U)[1]
    for k = 1:n-2
        b[k] = dot(U[k:k+2], x[k:k+2])
    end
    b[n-1] = dot(U[n-1,n-1:n],x[n-1:n])
    b[n] = U[n,n]*x[n]
    return b
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    # TODO: populate x so that U*x == b (up to rounding)
    n = size(U)[1]
    
    if length(b) != n
        error("The system is not compatible")
    end
    
    for k = n:-1:1  # start with k=n, then k=n-1, ...
        r = b[k]  # dummy variable
        for j = k+1:min(k+3,n)
            r -= U[k,j]*x[j]
        end
        x[k] = r/U[k,k]
    end
    x
end

\ (generic function with 150 methods)

### Permutation Matrices

Permutation matrices are matrices that represent the action of permuting the entries of a vector,
that is, matrix representations of the symmetric group $S_n$, acting on $ℝ^n$.

We can encode a permutation in vector $\mathbf σ = [σ_1,\ldots,σ_n]^⊤$. 
This induces an action on a vector (using indexing notation)
$$
𝐯[\mathbf σ] = \begin{bmatrix}v_{σ_1}\\ \vdots \\ v_{σ_n} \end{bmatrix}
$$


**Example (permutation of a vector)** 
Consider the permutation $σ$ given by
$$
\begin{pmatrix}
 1 & 2 & 3 & 4 & 5 \cr
 1 & 4 & 2 & 5 & 3
 \end{pmatrix}
$$
We can apply it to a vector:

In [18]:
σ = [1, 4, 2, 5, 3] # Permutation vec
v = [1, 2, 3, 4, 5] # Original vec
v[σ] 

5-element Vector{Int64}:
 1
 4
 2
 5
 3

Its inverse permutation $σ^{-1}$ has Cauchy notation coming from swapping the rows of
the Cauchy notation of $σ$ and sorting:
$$
\begin{pmatrix}
 1 & 4 & 2 & 5 & 3 \cr
 1 & 2 & 3 & 4 & 5
 \end{pmatrix} \rightarrow \begin{pmatrix}
 1 & 2 & 3 & 4 & 5 \cr
 1 & 3 & 5 & 2 & 4
 \end{pmatrix} 
$$
Julia has the function `invperm` for computing the vector that encodes
the inverse permutation:
And indeed:

In [19]:
σ⁻¹ = invperm(σ) # Compute inverse permutation

5-element Vector{Int64}:
 1
 3
 5
 2
 4

In [20]:
v[σ][σ⁻¹] # Restore back to the original vector

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [21]:
# Permutation matrix
P = I(5)[σ,:]

5×5 SparseArrays.SparseMatrixCSC{Bool, Int64} with 5 stored entries:
 1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅
 ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1
 ⋅  ⋅  1  ⋅  ⋅

In [22]:
P * v == v[σ]

true

### Question
Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.



In [23]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    if P.p[k] == j
        1
    else
        0
    end
end

function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    x[P.p]
end


p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
P

5×5 PermutationMatrix:
 1  0  0  0  0
 0  0  0  1  0
 0  1  0  0  0
 0  0  0  0  1
 0  0  1  0  0

### Reflection matrix

We can rotate an arbitrary vector to the unit axis without using trigonometric functions.

The matrix
$$Q = {1 \over \sqrt{a^2 + b^2}}\begin{bmatrix}
 a & b \cr -b & a
\end{bmatrix}
$$
is a rotation matrix satisfying
$$
Q \begin{bmatrix} a \\ b \end{bmatrix} = \sqrt{a^2 + b^2} \begin{bmatrix} 1 \\ 0 \end{bmatrix}
$$

**Reflection matrix**

Given a vector $v$ satisfying $||v|| = 1$, the reflection matrix is the orthogonal matrix  $$Q_v \triangleq I - 2vv^T$$. These are reflections in the direction of $v$.

We can apply $Q_v$ to a vector using the expression: $$Q_vx = x - 2v(v^Tx)$$

**Question** Now complete the implementation of a type representing reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [4]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    # TODO: Return Q[k,j]
    (k == j) - 2Q.v[k]*Q.v[j] # note true is treated like 1 and false like 0
end

function *(P::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
    x - 2*Q.v * dot(Q.v,x) # (Q.v'*x) also works instead of dot
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I

Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [1.0 4.336808689942018e-19 … -3.469446951953614e-18 0.0; 4.336808689942018e-19 1.0 … 7.589415207398531e-19 -8.673617379884035e-19; … ; -3.469446951953614e-18 7.589415207398531e-19 … 1.0 0.0; 0.0 -8.673617379884035e-19 … 0.0 1.0] ≈ UniformScaling{Bool}(true)

### Householder reflection
For a given vector $x$, define the Householder reflection $$Q^{\pm, H}_x := Q_w = I-2ww^T$$
for $y = \mp ||x||e_1 + x$ and $w = \frac{y}{\\y\\}$

The default choice in sign is $Q^H_x:=Q_x^{-sign(x1),H}$, which depends on the sign of the first entry of $x$ to gain stability.

And $$Q_x^{\pm,H}x = \pm ||x||e_1$$

Implementation is as follow:

In [24]:
function householderreflection(s::Bool, x::AbstractVector)
    y = copy(x) # don't modify `x`
    if s
        y[1] -= norm(x)
    else
        y[1] += norm(x)
    end
    Reflection(y)
end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ [norm(x);zeros(eltype(x),length(x)-1)]

Q = householderreflection(false, x)
@test Q isa Reflection
@test Q*x ≈ [-norm(x);zeros(eltype(x),length(x)-1)]

Test Passed
  Expression: Q * x ≈ [-(norm(x)); zeros(eltype(x), length(x) - 1)]
   Evaluated: [-2.062348692933986, 1.1102230246251565e-16, -8.326672684688674e-17, 1.6653345369377348e-16, 6.661338147750939e-16] ≈ [-2.0623486929339863, 0.0, 0.0, 0.0, 0.0]